In [2]:
import pandas as pd
import numpy as np

In [3]:
## read the Dataset
# https://www.kaggle.com/code/muhammedafsalpm/eda-fifa-21

In [4]:
data = pd.read_csv("FIFA21.csv")

In [5]:
#data

# Remove Non Essentital Columns



In [6]:
drops = {"N":['Name',"photoUrl",'playerUrl','Joined','Loan Date End',
              'Club','Positions','LongName','Nationality']}
data = data.drop(drops['N'], axis=1)
data = data.iloc[:, :18] # Select only the first 18 columns (index 0 to 17)
print(data.columns)

Index(['Unnamed: 0', 'ID', 'Age', '↓OVA', 'POT', 'Contract', 'Height',
       'Weight', 'Preferred Foot', 'BOV', 'Best Position', 'Value', 'Wage',
       'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy'],
      dtype='object')


In [7]:
# Remove Any Non Numeric Charecters 

In [8]:
data.dtypes

Unnamed: 0           int64
ID                   int64
Age                  int64
↓OVA                 int64
POT                  int64
Contract            object
Height              object
Weight              object
Preferred Foot      object
BOV                  int64
Best Position       object
Value               object
Wage                object
Release Clause      object
Attacking            int64
Crossing             int64
Finishing            int64
Heading Accuracy     int64
dtype: object

In [7]:
#data

In [8]:
converted_heights = []
for height in data['Height']:
    # Check if the height is in 'ft' and 'inches' format (e.g., "5'11\"")
    if "'" in height and "\"" in height:
        # Extract feet and inches from the string
        feet, inches = height.split("'")
        inches = inches.replace("\"", "")  # Remove the closing quote symbol
        # Convert to total inches
        total_inches = int(feet) * 12 + int(inches)
        # Convert inches to centimeters
        height_cm = total_inches * 2.54
    else:
        # Convert directly if height is already in cm (e.g., '180')
        height_cm = int(height.replace("cm", "").strip())
        
    converted_heights.append(height_cm)
data['Height'] = converted_heights

In [9]:
converted_weights = []
for i in data['Weight']:
    if 'lbs' in i:
        weight_value = ''.join(filter(str.isdigit, i))  # Remove non-numeric characters
        weight_kg = int(weight_value) * 0.453592  # Convert lbs to kg
    elif 'kg' in i:
        weight_value = ''.join(filter(str.isdigit, i))  # Remove non-numeric characters
        weight_kg = int(weight_value)  # Already in kg, just convert to integer
    converted_weights.append(weight_kg)

data['Weight'] = converted_weights

In [10]:
def clean_value(value):
    # Remove the '€' symbol and convert M (Million) or K (Thousand) to numeric
    if 'M' in value:
        return float(value.replace('€', '').replace('M', '')) * 1_000_000
    elif 'K' in value:
        return float(value.replace('€', '').replace('K', '')) * 1_000
    else:
        # Handle unexpected formats (optional)
        return float(value.replace('€', ''))

# Apply the function to the 'Value' column
data['Value'] = data['Value'].apply(clean_value)

In [11]:
def clean_wage(value):
    # Remove the '€' symbol and convert M (Million) or K (Thousand) to numeric
    if 'M' in value:
        return float(value.replace('€', '').replace('M', '')) * 1_000_000
    elif 'K' in value:
        return float(value.replace('€', '').replace('K', '')) * 1_000
    else:
        # Handle unexpected formats (optional)
        return float(value.replace('€', ''))

# Apply the function to the 'Value' column
data['Wage'] = data['Wage'].apply(clean_wage)

In [12]:
def clean_clause(value):
    # Remove the '€' symbol and convert M (Million) or K (Thousand) to numeric
    if 'M' in value:
        return float(value.replace('€', '').replace('M', '')) * 1_000_000
    elif 'K' in value:
        return float(value.replace('€', '').replace('K', '')) * 1_000
    else:
        # Handle unexpected formats (optional)
        return 0

# Apply the function to the 'Value' column
data['Release Clause'] = data['Release Clause'].apply(clean_clause)

In [9]:
data.dtypes

Best Position
CB     336
CAM    264
ST     250
GK     209
CDM    192
CM     164
RB     126
RM     124
LB     106
LM      90
LW      32
RW      29
CF      28
RWB     27
LWB     23
Name: count, dtype: int64

In [14]:
# Make the dataset friendly for one hot encoding

In [15]:
data['Preferred Foot'] = data['Preferred Foot'].map({'Left': 0, 'Right': 1})

In [16]:
data['Best Position'] = data['Best Position'].map({'ST': 1, 'CF': 2,'LW': 3, 'RW': 4,
                                                    'CAM': 5, 'CDM': 6,'CM': 7, 'LM': 8,
                                                    'RM': 9, 'GK': 0,'CB': 10, 'LB': 11,
                                                    'RB': 12, 'RWB': 13,'LWB': 14})

In [17]:
# Split columns if required. 

In [18]:
# Create new columns
data['start'] = np.nan
data['end'] = np.nan
data['loan'] = 0

# Function to process the Contract column
def process_contract(contract):
    if 'On Loan' in contract:
        # Handle "On Loan" entries
        parts = contract.split()
        start_year = int(parts[-3])  # Extract the start year
        return start_year, start_year + 2, 1
    elif '~' in contract:
        # Handle range entries
        start_year, end_year = map(int, contract.split('~'))
        return start_year, end_year, 0
    elif 'Free' in contract:
        return 0, 0, 0  # Default values for 'Free'
    else:
        # Handle standalone year
        year = int(contract.strip())
        return year, year + 2, 0

# Apply function and split values into the new columns
data[['start', 'end', 'loan']] = data['Contract'].apply(lambda x: pd.Series(process_contract(x)))

# Convert to integers for clarity
data['start'] = data['start'].astype(int)
data['end'] = data['end'].astype(int)
data['loan'] = data['loan'].astype(int)

In [19]:

data = data.drop(['Contract'], axis=1)


In [20]:
data

,Unnamed: 0,ID,Age,↓OVA,POT,Height,Weight,Preferred Foot,BOV,Best Position,Value,Wage,Release Clause,Attacking,Crossing,Finishing,Heading Accuracy,start,end,loan
0,0,158023,33,93,93,170.0,72.0,0,93,4,103500000.0,560000.0,138400000.0,429,85,95,70,2004,2021,0
1,1,20801,35,92,92,187.0,83.0,1,92,1,63000000.0,220000.0,75900000.0,437,84,95,90,2018,2022,0
2,2,200389,27,91,93,188.0,87.0,1,91,0,120000000.0,125000.0,159400000.0,95,13,11,15,2014,2023,0
3,3,192985,29,91,91,181.0,70.0,1,91,5,129000000.0,370000.0,161000000.0,407,94,82,55,2015,2023,0
4,4,190871,28,91,91,175.0,68.0,1,91,3,132000000.0,270000.0,166500000.0,408,85,87,62,2017,2022,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,226537,26,75,77,184.0,79.0,0,77,1,7000000.0,40000.0,14200000.0,313,35,74,71,2019,2024,0
1996,1996,234730,23,75,80,175.0,68.0,1,75,12,8000000.0,10000.0,0.0,289,72,45,64,2021,2023,1
1997,1997,215531,27,75,75,177.0,72.0,0,75,9,6000000.0,53000.0,0.0,321,78,62,42,2021,2023,1
1998,1998,202445,26,75,79,188.0,83.0,1,77,10,7000000.0,20000.0,16300000.0,208,34,22,73,2017,2021,0
